In [1]:
# In a notebook cell
!pip install -U spacy[transformers]
!python -m spacy download en_core_web_trf
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.8/795.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 93.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.9/237.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.0/734.0 kB 29.7 MB/s eta 0:00:00
✔ Downloa

In [10]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("sarahmiller137/distilbert-base-uncased-ft-ncbi-disease")
model = AutoModel.from_pretrained("sarahmiller137/distilbert-base-uncased-ft-ncbi-disease")


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [11]:
import spacy
from transformers import pipeline

# Load both models
nlp_general = spacy.load("en_core_web_trf")
medical_ner = pipeline(
    "ner",
    model="sarahmiller137/distilbert-base-uncased-ft-ncbi-disease",
    tokenizer="sarahmiller137/distilbert-base-uncased-ft-ncbi-disease",
    aggregation_strategy="simple"  # Groups B- and I- tags together
)



Device set to use cpu


In [12]:
doc = nlp_general("Show me all patients with diabetes over 50")
ner_results = medical_ner(doc.text)

In [13]:
for ent in doc.ents:
    print(ent.text, ent.label_)

50 DATE


In [14]:
print(ner_results)

[{'entity_group': 'LABEL_0', 'score': np.float32(0.99973536), 'word': 'show me all patients with', 'start': 0, 'end': 25}, {'entity_group': 'LABEL_1', 'score': np.float32(0.9906848), 'word': 'diabetes', 'start': 26, 'end': 34}, {'entity_group': 'LABEL_0', 'score': np.float32(0.99979), 'word': 'over 50', 'start': 35, 'end': 42}]


In [15]:
texts = [
    "Show active cancer diagnoses.",
    "Patients in Dubai with asthma diagnosed after 2010.",
    "Get male patients with active hypertension.",
    "Get all conditions recorded after 2020."
]

for text in texts:
  print(medical_ner(text))

[{'entity_group': 'LABEL_0', 'score': np.float32(0.99982727), 'word': 'show active', 'start': 0, 'end': 11}, {'entity_group': 'LABEL_1', 'score': np.float32(0.7772065), 'word': 'cancer', 'start': 12, 'end': 18}, {'entity_group': 'LABEL_0', 'score': np.float32(0.9999349), 'word': 'diagnoses.', 'start': 19, 'end': 29}]
[{'entity_group': 'LABEL_0', 'score': np.float32(0.999877), 'word': 'patients in dubai with', 'start': 0, 'end': 22}, {'entity_group': 'LABEL_1', 'score': np.float32(0.991668), 'word': 'asthma', 'start': 23, 'end': 29}, {'entity_group': 'LABEL_0', 'score': np.float32(0.9999081), 'word': 'diagnosed after 2010.', 'start': 30, 'end': 51}]
[{'entity_group': 'LABEL_0', 'score': np.float32(0.99980193), 'word': 'get male patients with', 'start': 0, 'end': 22}, {'entity_group': 'LABEL_1', 'score': np.float32(0.9548367), 'word': 'active', 'start': 23, 'end': 29}, {'entity_group': 'LABEL_2', 'score': np.float32(0.9538944), 'word': 'hypertension', 'start': 30, 'end': 42}, {'entity_gr

In [16]:
categorized_examples = {
    "basic_conditions": [
        "Find all patients with diabetes", # diabetes was identified
        "Patient has a history of type 2 diabetes mellitus", # was identified as label 1 and label 2 because they're all one thing
        "Active chronic kidney disease stage 3", # Label 2 as one sentence
        "Resolved COVID-19 infection from last month" # Label 1 and Label 2
    ],

    "severity_examples": [
        "Severe hypertension diagnosed in 2023", #only the disease was labeled by two labels, seems like I might merge them together as one sentence.
        "Mild chronic obstructive pulmonary disease, stable", # only disease and two labels
        "Moderate depression, currently active", # this wasn't recognized
        "Critical diabetic ketoacidosis requiring immediate intervention" # critical diabetic ketoacidosis was all recognized as two labels.
    ],

    "temporal_examples": [
        "New onset diabetes diagnosed today", # diabetes
        "Long-standing hypertension since 2015", #TRUE
        "Recent exacerbation of asthma last week",
        "Chronic condition with acute flare-up"
    ],

    "verification_status": [
        "Suspected pneumonia pending chest X-ray",
        "Confirmed diagnosis of appendicitis",
        "Provisional diagnosis of migraine headache",
        "Rule out myocardial infarction"
    ],

    "anatomical_specific": [
        "Left ventricular heart failure",
        "Right lower lobe pneumonia",
        "Anterior wall ST-elevation myocardial infarction",
        "Bilateral diabetic retinopathy" # all anatomical specifics get recognized with the disease as one label
    ],

    "multiple_conditions": [
        "Patient has diabetes, hypertension, and hyperlipidemia",
        "Comorbidities include heart failure and atrial fibrillation",
        "Past medical history significant for stroke, diabetes, and chronic kidney disease"
    ]
}

# Okay, so if label 1 is followed by label 2, it usually a continutaiton of the name of the disease.
# Anatomical locations are labeled with the disases but usually as label 1


In [19]:
for examples in categorized_examples.values():
  for example in examples:
    print([ent for ent in medical_ner(example) if ent['entity_group'] != 'LABEL_0'])

[{'entity_group': 'LABEL_1', 'score': np.float32(0.97493446), 'word': 'diabetes', 'start': 23, 'end': 31}]
[{'entity_group': 'LABEL_1', 'score': np.float32(0.99138767), 'word': 'type', 'start': 25, 'end': 29}, {'entity_group': 'LABEL_2', 'score': np.float32(0.993996), 'word': '2 diabetes mellitus', 'start': 30, 'end': 49}]
[{'entity_group': 'LABEL_2', 'score': np.float32(0.947604), 'word': 'chronic kidney disease', 'start': 7, 'end': 29}]
[{'entity_group': 'LABEL_1', 'score': np.float32(0.8078655), 'word': '##vid', 'start': 11, 'end': 14}, {'entity_group': 'LABEL_2', 'score': np.float32(0.91117895), 'word': '- 19', 'start': 14, 'end': 17}]
[{'entity_group': 'LABEL_1', 'score': np.float32(0.8764031), 'word': 'hyper', 'start': 7, 'end': 12}, {'entity_group': 'LABEL_2', 'score': np.float32(0.8162907), 'word': '##tension', 'start': 12, 'end': 19}]
[{'entity_group': 'LABEL_1', 'score': np.float32(0.5894594), 'word': 'chronic', 'start': 5, 'end': 12}, {'entity_group': 'LABEL_2', 'score': np.

In [20]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


[{'sequence': 'List conditions for patient with ID 12345.', 'labels': ['Patient', 'Conditions', 'Others'], 'scores': [0.5614208579063416, 0.4316323399543762, 0.006946844048798084]}, {'sequence': 'Find diabetes cases diagnosed before 2015.', 'labels': ['Patient', 'Conditions', 'Others'], 'scores': [0.48120564222335815, 0.36231529712677, 0.15647904574871063]}, {'sequence': 'Patients in Dubai with asthma diagnosed after 2010.', 'labels': ['Patient', 'Conditions', 'Others'], 'scores': [0.5967134833335876, 0.3543432354927063, 0.04894327372312546]}, {'sequence': 'Show patients named Fatima, born in 1995, living in Cairo.', 'labels': ['Patient', 'Conditions', 'Others'], 'scores': [0.9529616832733154, 0.025991203263401985, 0.02104710228741169]}, {'sequence': 'List patients living in Khartoum, Sudan.', 'labels': ['Patient', 'Conditions', 'Others'], 'scores': [0.811912477016449, 0.14667534828186035, 0.041412197053432465]}]
[{'sequence': 'List conditions for patient with ID 12345.', 'labels': ['P

KeyboardInterrupt: 

In [22]:
import pandas as pd

This is a great working Text classification using Zero-shot classification

In [24]:
texts = ["List conditions for patient with ID 12345.",
 "Find diabetes cases diagnosed before 2015.",
 "Patients in Dubai with asthma diagnosed after 2010.",
 "Show patients named Fatima, born in 1995, living in Cairo.",
 "List patients living in Khartoum, Sudan.",
 "Provisional diagnosis of migraine headache",
 "Show me all patients with diabetes over 50",
 ]

possible_options = ["medical conditions", "patient information", "others"]

for text in texts:
  results = classifier(text, possible_options)
  labels_idx_max = pd.Series(results['scores']).idxmax()
  print(results['labels'][labels_idx_max])

patient information
medical conditions
patient information
patient information
patient information
medical conditions
patient information


I just need to add the following into my entity ruler to get a simple working prototype:

- **Severity:** mild, moderate, severe, critical
- **Clinical status:** active, resolved, inactive, remission
- **Verification status:** suspected, confirmed, provisional
- **Onset timing:** acute, chronic, new onset, long-standing
- **Body sites:** left ventricular, right lower lobe, bilateral -> This is already catched with the medical NER model I have, but. I need to diffrentitate it from the text.
- **Dates/temporal info:** "since 2015", "last week", "today"

I'll try my best to cover most cases using entity ruler.

For now let me focus on building a framework in which I can add to it later specific entity rulers related to conditions. Also the zero-shot classification seems like an easy solution to classifiy requests later to GET, POST, PATCH, DELETE